In [255]:
import numpy as np
import pandas as pd

# import nltk
# # nltk.download('stopwords')
# # nltk.download('omw-1.4')
# # nltk.download('wordnet')
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.calibration
import sklearn.preprocessing
import matplotlib.pyplot as plt

# READ DATA & CLEANING - metadata_team
### 1. READ DATA
### 2. Generate Yearly Window & 2 Years Window & 5 Years Window
##### 2.1. CALCULATE YEARLY WINDOW (SUMMATION OF ALL CITATIONS)
##### 2.2. CALCULATE 2 YEARS WINDOW (PUBLISH YEAR - PUBLISH YEAR + 2)
##### 2.3. CALCULATE 5 YEARS WINDOW (PUBLISH YEAR - PUBLISH YEAR + 5)
### 3. DROP 'NAN'S IN THE COLUMNS ['PY'],['UNIQUE NATIONALITIES']
##### 3.1 SELECT DATA RELATED TO SDG 7 - ENERGY & SEPARATE THE UNIQUE NATIONALITIES
##### 3.2 NULL CHECK

In [256]:
data = pd.read_pickle('data/metadata_team.pkl')
pd.set_option('display.max_columns', 15)
data.head(4)

,PT,AU,BA,BE,GP,AF,BF,...,2017.0,2018.0,2019.0,2020.0,2021.0,Nationalities,Unique Nationalities
0,J,"Sztumski, W",NaN,NaN,NaN,"Sztumski, Wieslaw",NaN,...,0.0,0.0,1.0,0.0,0.0,[[ ]],[[ ]]
1,J,"Degai, TS; Petrov, AN",NaN,NaN,NaN,"Degai, Tatiana S.; Petrov, Andrey N.",NaN,...,0,0,0,0,1,"[Russia, Russia, USA, Russia]","[Russia, USA]"
2,S,"Kanapathy, S; Lee, KE; Mokhtar, M; Sivapalan, ...",NaN,"Lee, KE",NaN,"Kanapathy, Suganty; Lee, Khai Ern; Mokhtar, Ma...",NaN,...,0.0,0.0,0.0,0.0,1.0,"[Malaysia, Malaysia, Malaysia, Malaysia]",[Malaysia]
3,J,"Selvi, M; Selvi, M",NaN,NaN,NaN,"Selvi, M; Selvi, M",NaN,...,0.0,0.0,0.0,0.0,0.0,[Turkey],[Turkey]


In [257]:
pd.set_option('display.max_columns', None)
mydata = data[data.PY >= 1901.0]
mydata = mydata.reset_index(drop=True)
# mydata['Unique Nationalities'] = mydata['Unique Nationalities'].astype(str) 
# mydata = mydata[mydata['Unique Nationalities'].str.contains("[[' ']]", regex=False) == False]
mydata.head(5)

,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,SE,BS,LA,DT,CT,CY,CL,SP,HO,DE,ID,AB,C1,RP,EM,RI,OI,FU,FX,CR,NR,TC,Z9,U1,U2,PU,PI,PA,SN,EI,BN,J9,JI,PD,PY,VL,IS,PN,SU,SI,MA,BP,EP,AR,DI,D2,EA,PG,WC,SC,GA,UT,PM,OA,HC,HP,DA,Column1,1901.0,1902.0,1903.0,1904.0,1905.0,1906.0,1907.0,1908.0,1909.0,1910.0,1911.0,1912.0,1913.0,1914.0,1915.0,1916.0,1917.0,1918.0,1919.0,1920.0,1921.0,1922.0,1923.0,1924.0,1925.0,1926.0,1927.0,1928.0,1929.0,1930.0,1931.0,1932.0,1933.0,1934.0,1935.0,1936.0,1937.0,1938.0,1939.0,1940.0,1941.0,1942.0,1943.0,1944.0,1945.0,1946.0,1947.0,1948.0,1949.0,1950.0,1951.0,1952.0,1953.0,1954.0,1955.0,1956.0,1957.0,1958.0,1959.0,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,Nationalities,Unique Nationalities
0,J,"Sztumski, W",NaN,NaN,NaN,"Sztumski, Wieslaw",NaN,NaN,The Mythology of Sustainable Development,PROBLEMY EKOROZWOJU,NaN,NaN,English,Article,NaN,NaN,NaN,NaN,NaN,sustainable development; ideology of sustainab...,NaN,In this article I present a proposition that a...,NaN,"Sztumski, W (corresponding author), Ul Okolna ...",ws34@op.pl,NaN,NaN,NaN,NaN,NaN,6.0,10.0,10.0,0.0,4.0,POLITECHNIKA LUBELSKA,LUBLIN,"WYDZIAL INZYNIERII SRODOWSKA, UL NADBYSTRZYCKA...",1895-6912,NaN,NaN,PROBL EKOROZW,Probl. Ekorozw.,NaN,2009.0,4,2,NaN,NaN,NaN,NaN,13.0,23,NaN,NaN,NaN,NaT,11.0,Environmental Studies,Environmental Sciences & Ecology,465WN,WOS:000267620200002,NaN,NaN,NaN,NaN,2021-09-21,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,[[ ]],[[ ]]
1,J,"Degai, TS; Petrov, AN",NaN,NaN,NaN,"Degai, Tatiana S.; Petrov, Andrey N.",NaN,NaN,Rethinking Arctic sustainable development agen...,INTERNATIONAL JOURNAL OF SUSTAINABLE DEVELOPME...,NaN,NaN,English,Article,NaN,NaN,NaN,NaN,NaN,sustainable development; Arctic; Indigenous pe...,NaN,The year 2020 has urged Humanity to rethink th...,"[Degai, Tatiana S.] Council Itelmens Kamchatka...","Degai, TS (corresponding author), Council Itel...",tatiana.degai@uni.edu,NaN,NaN,"NSFNational Science Foundation (NSF) [1441381,...",This work has been partially supported by NSF ...,NaN,46.0,1.0,1.0,8.0,9.0,TAYLOR & FRANCIS INC,PHILADELPHIA,"530 WALNUT STREET, STE 850, PHILADELPHIA, PA 1...",1350-4509,1745-2627,NaN,INT J SUST DEV WORLD,Int. J. Sustain. Dev. World Ecol.,AUG 18,2021.0,28,6,NaN,NaN,NaN,NaN,518.0,523,NaN,10.1080/13504509.2020.1868608,NaN,2021-01-01,6.0,Green & Sustainable Science & Technology; Ecology,Science & Technology - Other Topics; Environme...,TR7CK,WOS:000608316000001,NaN,Bronze,NaN,NaN,2021-09-21,NaN,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,"[Russia, Russia, USA, Russia]","[Russia, USA]"
2,S,"Kanapathy, S; Lee, KE; Mokhtar, M; Sivapalan, ...",NaN,"Lee, KE",NaN,"Kanapathy, Suganty; Lee, Khai Ern; Mokhtar, Ma...",NaN,NaN,Enculturing Sustainable Development Concept Th...,CONCEPTS AND APPROACHES FOR SUSTAINABILITY MAN...,Advances in Science Technology & Innovation,NaN,English,Article; Book Chapter,NaN,NaN,NaN,NaN,NaN,Education for sustainable development; Chemist...,ENVIRONMENTAL KNOWLEDGE; DEV

### 2. Generate Yearly Window & 2 Years Window & 5 Years Window

In [258]:
df_cols = np.array(mydata.columns.to_list())
ind_yr1 = np.where(df_cols == '1901.0')[0][0]
ind_yrlast = np.where(df_cols == '2021.0')[0][0]

yearlycitation = np.sum(mydata.iloc[:,ind_yr1:ind_yrlast], axis = 1)

In [259]:
publishyear = mydata['PY']
twoyearwindow = mydata['PY'] + 2
# fiveyearwindow = mydata['PY'] + 5

In [260]:
twoyearcitation = []
# fiveyearcitation = []

In [261]:
for i in range (len(mydata)):
    if twoyearwindow[i] >= 2019.0:
        twoyearwindow[i] = 2021

In [262]:
for i in range (len(mydata)):
    temp = []
    start = int(publishyear[i])
    end = int(twoyearwindow[i])
    for j in range (start, end+1):
        temp.append(mydata[j][i])
    twoyearcitation.append(sum(temp))

In [263]:
mydata['yearly window'] = yearlycitation
mydata['2 years window'] = twoyearcitation

In [264]:
mydata.head()

,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,SE,BS,LA,DT,CT,CY,CL,SP,HO,DE,ID,AB,C1,RP,EM,RI,OI,FU,FX,CR,NR,TC,Z9,U1,U2,PU,PI,PA,SN,EI,BN,J9,JI,PD,PY,VL,IS,PN,SU,SI,MA,BP,EP,AR,DI,D2,EA,PG,WC,SC,GA,UT,PM,OA,HC,HP,DA,Column1,1901.0,1902.0,1903.0,1904.0,1905.0,1906.0,1907.0,1908.0,1909.0,1910.0,1911.0,1912.0,1913.0,1914.0,1915.0,1916.0,1917.0,1918.0,1919.0,1920.0,1921.0,1922.0,1923.0,1924.0,1925.0,1926.0,1927.0,1928.0,1929.0,1930.0,1931.0,1932.0,1933.0,1934.0,1935.0,1936.0,1937.0,1938.0,1939.0,1940.0,1941.0,1942.0,1943.0,1944.0,1945.0,1946.0,1947.0,1948.0,1949.0,1950.0,1951.0,1952.0,1953.0,1954.0,1955.0,1956.0,1957.0,1958.0,1959.0,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,Nationalities,Unique Nationalities,yearly window,2 years window
0,J,"Sztumski, W",NaN,NaN,NaN,"Sztumski, Wieslaw",NaN,NaN,The Mythology of Sustainable Development,PROBLEMY EKOROZWOJU,NaN,NaN,English,Article,NaN,NaN,NaN,NaN,NaN,sustainable development; ideology of sustainab...,NaN,In this article I present a proposition that a...,NaN,"Sztumski, W (corresponding author), Ul Okolna ...",ws34@op.pl,NaN,NaN,NaN,NaN,NaN,6.0,10.0,10.0,0.0,4.0,POLITECHNIKA LUBELSKA,LUBLIN,"WYDZIAL INZYNIERII SRODOWSKA, UL NADBYSTRZYCKA...",1895-6912,NaN,NaN,PROBL EKOROZW,Probl. Ekorozw.,NaN,2009.0,4,2,NaN,NaN,NaN,NaN,13.0,23,NaN,NaN,NaN,NaT,11.0,Environmental Studies,Environmental Sciences & Ecology,465WN,WOS:000267620200002,NaN,NaN,NaN,NaN,2021-09-21,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,[[ ]],[[ ]],10.0,6.0
1,J,"Degai, TS; Petrov, AN",NaN,NaN,NaN,"Degai, Tatiana S.; Petrov, Andrey N.",NaN,NaN,Rethinking Arctic sustainable development agen...,INTERNATIONAL JOURNAL OF SUSTAINABLE DEVELOPME...,NaN,NaN,English,Article,NaN,NaN,NaN,NaN,NaN,sustainable development; Arctic; Indigenous pe...,NaN,The year 2020 has urged Humanity to rethink th...,"[Degai, Tatiana S.] Council Itelmens Kamchatka...","Degai, TS (corresponding author), Council Itel...",tatiana.degai@uni.edu,NaN,NaN,"NSFNational Science Foundation (NSF) [1441381,...",This work has been partially supported by NSF ...,NaN,46.0,1.0,1.0,8.0,9.0,TAYLOR & FRANCIS INC,PHILADELPHIA,"530 WALNUT STREET, STE 850, PHILADELPHIA, PA 1...",1350-4509,1745-2627,NaN,INT J SUST DEV WORLD,Int. J. Sustain. Dev. World Ecol.,AUG 18,2021.0,28,6,NaN,NaN,NaN,NaN,518.0,523,NaN,10.1080/13504509.2020.1868608,NaN,2021-01-01,6.0,Green & Sustainable Science & Technology; Ecology,Science & Technology - Other Topics; Environme...,TR7CK,WOS:000608316000001,NaN,Bronze,NaN,NaN,2021-09-21,NaN,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,"[Russia, Russia, USA, Russia]","[Russia, USA]",0.0,1.0
2,S,"Kanapathy, S; Lee, KE; Mokhtar, M; Sivapalan, ...",NaN,"Lee, KE",NaN,"Kanapathy, Suganty; Lee, Khai Ern; Mokhtar, Ma...",NaN,NaN,Enculturing Sustainable Development Concept Th...,CONCEPTS AND APPROACHES FOR SUSTAINABILITY MAN...,Advances in Science Technology & Innovation,NaN,English,Article; Book Chapter,NaN,NaN,NaN,NaN,NaN,Education for sustainable develo

### 3. DROP 'NAN'S IN THE COLUMNS ['PY'],['UNIQUE NATIONALITIES']

In [265]:
mydata['Unique Nationalities'] = mydata['Unique Nationalities'].astype(str) 
mydata = mydata[mydata['Unique Nationalities'].str.contains("[[' ']]", regex=False) == False]

# removing rows where AB is nan/null
mydata = mydata.reset_index()

mydata = mydata[mydata["2 years window"] != 0]

#dropping rows where total citations is 0.0
mydata = mydata[mydata["yearly window"] != 0]

mydata['PY'] = mydata['PY'].astype(int)

mydata = mydata[mydata['AB'].notna()]

mydata['AB'] = mydata['AB'].str.lower()

mydata['TI'] = mydata['TI'].str.lower()

mydata['DE'] = mydata['DE'].str.lower()

mydata.head(4)

,index,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,SE,BS,LA,DT,CT,CY,CL,SP,HO,DE,ID,AB,C1,RP,EM,RI,OI,FU,FX,CR,NR,TC,Z9,U1,U2,PU,PI,PA,SN,EI,BN,J9,JI,PD,PY,VL,IS,PN,SU,SI,MA,BP,EP,AR,DI,D2,EA,PG,WC,SC,GA,UT,PM,OA,HC,HP,DA,Column1,1901.0,1902.0,1903.0,1904.0,1905.0,1906.0,1907.0,1908.0,1909.0,1910.0,1911.0,1912.0,1913.0,1914.0,1915.0,1916.0,1917.0,1918.0,1919.0,1920.0,1921.0,1922.0,1923.0,1924.0,1925.0,1926.0,1927.0,1928.0,1929.0,1930.0,1931.0,1932.0,1933.0,1934.0,1935.0,1936.0,1937.0,1938.0,1939.0,1940.0,1941.0,1942.0,1943.0,1944.0,1945.0,1946.0,1947.0,1948.0,1949.0,1950.0,1951.0,1952.0,1953.0,1954.0,1955.0,1956.0,1957.0,1958.0,1959.0,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,Nationalities,Unique Nationalities,yearly window,2 years window
4,5,J,"Agbedahin, AV",NaN,NaN,NaN,"Agbedahin, Adesuwa Vanessa",NaN,NaN,"sustainable development, education for sustain...",SUSTAINABLE DEVELOPMENT,NaN,NaN,English,Article,NaN,NaN,NaN,NaN,NaN,2030 education agenda; education; education fo...,SOUTHERN AFRICA,this paper aims to present a narrative of the ...,"[Agbedahin, Adesuwa Vanessa] Univ Free State, ...","Agbedahin, AV (corresponding author), Univ Fre...",advanagbedahin@gmail.com,"Agbedahin, Adesuwa Vanessa/AAP-2193-2021","Agbedahin, Adesuwa Vanessa/0000-0002-6271-6691",NaN,NaN,NaN,74.0,23.0,23.0,25.0,42.0,WILEY,HOBOKEN,"111 RIVER ST, HOBOKEN 07030-5774, NJ USA",0968-0802,1099-1719,NaN,SUSTAIN DEV,Sustain. Dev.,JUL,2019,27,4,NaN,NaN,NaN,NaN,669.0,680,NaN,10.1002/sd.1931,NaN,NaT,12.0,Development Studies; Green & Sustainable Scien...,Development Studies; Science & Technology - Ot...,IR4ZF,WOS:000481442100009,NaN,NaN,NaN,NaN,2021-09-21,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,11.0,10.0,"[Africa, Africa]",['Africa'],15.0,25.0
7,8,J,"Holden, E; Linnerud, K; Banister, D",NaN,NaN,NaN,"Holden, Erling; Linnerud, Kristin; Banister, D...",NaN,NaN,the imperatives of sustainable development,SUSTAINABLE DEVELOPMENT,NaN,NaN,English,Article,NaN,NaN,NaN,NaN,NaN,sustainable development; sustainable developme...,CAPABILITY APPROACH; GOALS; ENVIRONMENT; NEED,the united nations sustainable development goa...,"[Holden, Erling] Sogn & Fjordane Univ Coll, De...","Holden, E (corresponding author), Sogn & Fjord...",erling.holden@hisf.no,NaN,NaN,NaN,NaN,NaN,79.0,112.0,114.0,19.0,111.0,WILEY,HOBOKEN,"111 RIVER ST, HOBOKEN 07030-5774, NJ USA",0968-0802,1099-1719,NaN,SUSTAIN DEV,Sustain. Dev.,MAY-JUN,2017,25,3,NaN,NaN,NaN,NaN,213.0,226,NaN,10.1002/sd.1647,NaN,NaT,14.0,Development Studies; Green & Sustainable Scien...,Development Studies; Science & Technology - Ot...,FB1YV,WOS:000405941000005,NaN,"Green Published, Green Submitted, Green Accepted",Y,N,2021-09-21,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,30.0,27.0,37.0,15.0,"[England, Norway, Norway,

##### 3.1 SELECT DATA RELATED TO SDG 7 - ENERGY & SEPARATE THE UNIQUE NATIONALITIES

In [266]:
mydata.shape

(36934, 194)

In [267]:
sdg7 = "energy|electricity" #because sdg7

only_AB = mydata[mydata['AB'].notna() | mydata['DE'].notna() | mydata['TI'].notna()] #drop NANs in DE column to be able to do string match (n went from 80672 -> 70121)

only_AB = only_AB[only_AB['AB'].str.contains(sdg7, na = False) | only_AB['TI'].str.contains(sdg7, na = False) | only_AB['DE'].str.contains(sdg7, na = False)]

only_AB = only_AB[only_AB.PY != 2021]

In [268]:
only_AB.shape

(7538, 194)

In [269]:
import re

sdg7_new = pd.DataFrame(columns = ['PY', 'Unique Nationalities', 'yearly window', '2 years window']) #new dataframe to add new rws per nationlity to

count_dropped = 0

for i in range(len(only_AB)):
    uniq_nat = only_AB.iloc[i]['Unique Nationalities'] #get countries from column
    comm_add = re.sub("\s+", ",", uniq_nat.strip()) #replace space with comma
    joint = ''.join(str(e) for e in comm_add) #join all elements in string together
    nat_ls = re.findall(r"'([^']*)'", joint) #only get elements between '' and now has the countries in a list
    nat_ls = [x.lower() for x in nat_ls]
    
    #get other data
    py = only_AB.iloc[i]['PY']
    citations = only_AB.iloc[i]['yearly window']
    citations_2 = only_AB.iloc[i]['2 years window']
    
    if len(nat_ls) > 1: #if more than one country in list
        #drop row
#         for j in range(len(nat_ls)): #add row for each country, keeping py and citations same
#             new_row = {'PY': py, 'Unique Nationalities': nat_ls[j], 'citations sum': citations, '2 years window': citations_2}
#             sdg7_new = sdg7_new.append(new_row, ignore_index = True)
        count_dropped += 1
            
    elif len(nat_ls) == 1: #if only one country in list
        new_row = {'PY': py, 'Unique Nationalities': nat_ls[0], 'citations sum': citations, '2 years window': citations_2} #just add one row
        sdg7_new = sdg7_new.append(new_row, ignore_index = True)
    else:
        count_dropped += 1
        
print("Rows Dropped because of country collaboration research:", count_dropped, "/", only_AB.shape[0])
print("Ratio of rows dropped because of country collaboration research:", (count_dropped / only_AB.shape[0]) * 100, "%")
sdg7_new = sdg7_new[~sdg7_new['PY'].isin(['2021'])]
sdg7_new = sdg7_new.drop('yearly window', axis=1)

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

...

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

Rows Dropped because of country collaboration research: 2118 / 7538
Ratio of rows dropped because of country collaboration research: 28.09763863093659 %


/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdg7_new = sdg7_new.append(new_row, ignore_index = True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/857566162.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

In [270]:
sdg7_new

,PY,Unique Nationalities,2 years window,citations sum
0,2019,poland,9.0,5.0
1,2017,russia,13.0,10.0
2,2015,korea,11.0,31.0
3,2018,poland,2.0,2.0
4,2012,pakistan,2.0,2.0
...,...,...,...,...
5415,2018,italy,5.0,5.0
5416,2010,italy,1.0,7.0
5417,2019,pakistan,7.0,2.0
5418,2020,finland,1.0,1.0


In [271]:
mydata[mydata['Unique Nationalities'].str.contains('Republic')]

,index,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,SE,BS,LA,DT,CT,CY,CL,SP,HO,DE,ID,AB,C1,RP,EM,RI,OI,FU,FX,CR,NR,TC,Z9,U1,U2,PU,PI,PA,SN,EI,BN,J9,JI,PD,PY,VL,IS,PN,SU,SI,MA,BP,EP,AR,DI,D2,EA,PG,WC,SC,GA,UT,PM,OA,HC,HP,DA,Column1,1901.0,1902.0,1903.0,1904.0,1905.0,1906.0,1907.0,1908.0,1909.0,1910.0,1911.0,1912.0,1913.0,1914.0,1915.0,1916.0,1917.0,1918.0,1919.0,1920.0,1921.0,1922.0,1923.0,1924.0,1925.0,1926.0,1927.0,1928.0,1929.0,1930.0,1931.0,1932.0,1933.0,1934.0,1935.0,1936.0,1937.0,1938.0,1939.0,1940.0,1941.0,1942.0,1943.0,1944.0,1945.0,1946.0,1947.0,1948.0,1949.0,1950.0,1951.0,1952.0,1953.0,1954.0,1955.0,1956.0,1957.0,1958.0,1959.0,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,Nationalities,Unique Nationalities,yearly window,2 years window
201,209,C,"Dvorakova, L; Zborkova, J",NaN,"Katalinic, B",NaN,"Dvorakova, Lilia; Zborkova, Jitka",NaN,NaN,integration of sustainable development at ente...,24TH DAAAM INTERNATIONAL SYMPOSIUM ON INTELLIG...,Procedia Engineering,NaN,English,Proceedings Paper,24th DAAAM International Symposium on Intellig...,"OCT 23-26, 2013","Univ Zadar, Zadar, CROATIA","DAAAM Int Assoc, Int Acad Engn, Vienna Univ Te...",Univ Zadar,sustainable development; environment; business...,NaN,the goal of this paper is to present the resul...,"[Dvorakova, Lilia; Zborkova, Jitka] Univ W Boh...","Zborkova, J (corresponding author), Univ W Boh...",prochaz@kfu.zcu.cz,NaN,"Zborkova, Jitka/0000-0003-2716-1536",NaN,NaN,NaN,10.0,18.0,18.0,0.0,4.0,ELSEVIER SCIENCE BV,AMSTERDAM,"SARA BURGERHARTSTRAAT 25, PO BOX 211, 1000 AE ...",1877-7058,NaN,NaN,PROCEDIA ENGINEER,NaN,NaN,2014,69,NaN,NaN,NaN,NaN,NaN,686.0,695,NaN,10.1016/j.proeng.2014.03.043,NaN,NaT,10.0,"Automation & Control Systems; Engineering, Mul...",Automation & Control Systems; Engineering,BA4JN,WOS:000335859300092,NaN,gold,NaN,NaN,2021-09-21,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,3.0,2.0,5.0,4.0,"[Republic, Republic]",['Republic'],15.0,3.0
373,385,J,"Hak, T; Janouskova, S; Moldan, B",NaN,NaN,NaN,"Hak, Tomas; Janouskova, Svatava; Moldan, Bedrich",NaN,NaN,sustainable development goals: a need for rele...,ECOLOGICAL INDICATORS,NaN,NaN,English,Article,NaN,NaN,NaN,NaN,NaN,sustainable development goals; indicatorsi rel...,SCIENCE,at the un in new york the open working group c...,"[Hak, Tomas; Janouskova, Svatava; Moldan, Bedr...","Janouskova, S (corresponding author), Charles ...",svatava.janouskova@czp.cuni.cz,"Hak, Tomas/R-1845-2016; Moldan, Bedrich/P-4545...","Hak, Tomas/0000-0001-5271-4687; Moldan, Bedric...",Czech Science FoundationGrant Agency of the Cz...,The work presented in this article was underta...,NaN,66.0,338.0,346.0,29.0,299.0,ELSEVIER,AMSTERDAM,"RADARWEG 29, 1043 NX AMSTERDAM, NETHERLANDS",1470-160X,1872-7034,NaN,ECOL INDIC,Ecol. Indic.,JAN,2016,60,NaN,NaN,NaN,NaN,NaN,565.0,573,NaN,10.1016/j.ecolind.2015.08.003,NaN,NaT,9.0,Biodiversity Conservation; Environmental Sciences,Biodiversity & Conservation; Environmental Sci...,CZ9GW,WOS:000367407000058,NaN,NaN,Y,N,2021-09-21,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [272]:
mydata[(mydata['PY'] == 2007) & (mydata['yearly window'] == 134.0)]

,index,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,SE,BS,LA,DT,CT,CY,CL,SP,HO,DE,ID,AB,C1,RP,EM,RI,OI,FU,FX,CR,NR,TC,Z9,U1,U2,PU,PI,PA,SN,EI,BN,J9,JI,PD,PY,VL,IS,PN,SU,SI,MA,BP,EP,AR,DI,D2,EA,PG,WC,SC,GA,UT,PM,OA,HC,HP,DA,Column1,1901.0,1902.0,1903.0,1904.0,1905.0,1906.0,1907.0,1908.0,1909.0,1910.0,1911.0,1912.0,1913.0,1914.0,1915.0,1916.0,1917.0,1918.0,1919.0,1920.0,1921.0,1922.0,1923.0,1924.0,1925.0,1926.0,1927.0,1928.0,1929.0,1930.0,1931.0,1932.0,1933.0,1934.0,1935.0,1936.0,1937.0,1938.0,1939.0,1940.0,1941.0,1942.0,1943.0,1944.0,1945.0,1946.0,1947.0,1948.0,1949.0,1950.0,1951.0,1952.0,1953.0,1954.0,1955.0,1956.0,1957.0,1958.0,1959.0,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,Nationalities,Unique Nationalities,yearly window,2 years window
33144,35869,J,"Behrens, A; Giljum, S; Kovanda, J; Niza, S",NaN,NaN,NaN,"Behrens, Arno; Giljum, Stefan; Kovanda, Jan; N...",NaN,NaN,the material basis of the global economy world...,ECOLOGICAL ECONOMICS,NaN,NaN,English,Article,NaN,NaN,NaN,NaN,NaN,environmental policy; global resource use; int...,EUROPEAN-UNION; GROWTH,material flow accounting and analysis (mfa) ha...,"[Kovanda, Jan] Charles Univ Prague, Ctr Enviro...","Behrens, A (corresponding author), Garnisongas...",arno.behrens@seri.at,"Giljum, Stefan/M-3162-2015; Kovanda, Jan/P-438...","Giljum, Stefan/0000-0002-4719-5867; Kovanda, J...",NaN,NaN,NaN,56.0,145.0,151.0,0.0,81.0,ELSEVIER,AMSTERDAM,"RADARWEG 29, 1043 NX AMSTERDAM, NETHERLANDS",0921-8009,1873-6106,NaN,ECOL ECON,Ecol. Econ.,DEC 15,2007,64,2,NaN,NaN,NaN,NaN,444.0,453,NaN,10.1016/j.ecolecon.2007.02.034,NaN,NaT,10.0,Ecology; Economics; Environmental Sciences; En...,Environmental Sciences & Ecology; Business & E...,249XT,WOS:000252264600020,NaN,NaN,NaN,NaN,2021-09-22,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,5.0,10.0,10.0,13.0,5.0,11.0,12.0,19.0,18.0,11.0,14.0,11.0,"[Republic, Republic]",['Republic'],134.0,6.0


In [273]:
sdg7_new[sdg7_new['Unique Nationalities'] == 'republic']

,PY,Unique Nationalities,2 years window,citations sum
47,2020,republic,18.0,10.0
135,2016,republic,1.0,1.0
164,2014,republic,1.0,2.0
825,2019,republic,10.0,6.0
893,2016,republic,1.0,1.0
1174,2019,republic,8.0,6.0
1291,2019,republic,7.0,6.0
1466,2014,republic,1.0,4.0
1528,2014,republic,1.0,1.0
1724,2014,republic,3.0,7.0


In [274]:
sdg7_new.drop(sdg7_new[sdg7_new['Unique Nationalities'] == '94720'].index, inplace = True)
sdg7_new.drop(sdg7_new[sdg7_new['Unique Nationalities'] == '08544'].index, inplace = True)
sdg7_new.drop(sdg7_new[sdg7_new['Unique Nationalities'] == '20433'].index, inplace = True)
sdg7_new.loc[sdg7_new['Unique Nationalities'] == '263001,india', 'Unique Nationalities'] = 'india'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == '110067,india', 'Unique Nationalities'] = 'india'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'res,vienna,austria', 'Unique Nationalities'] = 'austria'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'stockholm,sweden', 'Unique Nationalities'] = 'sweden'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'milan,italy', 'Unique Nationalities'] = 'italy'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == '4yn,england', 'Unique Nationalities'] = 'united kingdom'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'usa', 'Unique Nationalities'] = 'united states'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'wales', 'Unique Nationalities'] = 'united kingdom'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'scotland', 'Unique Nationalities'] = 'united kingdom'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'england', 'Unique Nationalities'] = 'united kingdom'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'herceg', 'Unique Nationalities'] = 'montenegro'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'lanka', 'Unique Nationalities'] = 'sri lanka'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'paulo,brazil', 'Unique Nationalities'] = 'brazil'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'delft,netherlands', 'Unique Nationalities'] = 'netherlands'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'freiburg,germany', 'Unique Nationalities'] = 'germany'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'pradesh,india', 'Unique Nationalities'] = 'india'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'freiburg,germany', 'Unique Nationalities'] = 'germany'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == '13,france', 'Unique Nationalities'] = 'france'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == '7ay,avon,england', 'Unique Nationalities'] = 'united kingdom'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'korea', 'Unique Nationalities'] = 'korea, rep.'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'africa', 'Unique Nationalities'] = 'south africa'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'turkey', 'Unique Nationalities'] = 'turkiye'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'republic', 'Unique Nationalities'] = 'czechia'
sdg7_new.loc[sdg7_new['Unique Nationalities'] == 'brussels,belgium', 'Unique Nationalities'] = 'belgium'

In [275]:
len(sdg7_new['Unique Nationalities'].unique())

102

In [276]:
sdg7_new['Unique Nationalities'].unique()

array(['poland', 'russia', 'korea, rep.', 'pakistan', 'luxembourg',
       'united states', 'turkiye', 'iran', 'japan', 'canada', 'brazil',
       'romania', 'china', 'germany', 'taiwan', 'denmark', 'lithuania',
       'switzerland', 'france', 'greece', 'india', 'arabia', 'austria',
       'australia', 'malaysia', 'czechia', 'united kingdom', 'finland',
       'spain', 'slovenia', 'zealand', 'algeria', 'netherlands', 'egypt',
       'croatia', 'ireland', 'latvia', 'nigeria', 'south africa', 'italy',
       'kosovo', 'portugal', 'morocco', 'serbia', 'iceland', 'macedonia',
       'mexico', 'fiji', 'sweden', 'thailand', 'slovakia', 'ukraine',
       'emirates', 'azerbaijan', 'norway', 'belgium', 'jordan',
       'bangladesh', 'kuwait', 'estonia', 'faso', 'ethiopia', 'ghana',
       'lebanon', 'colombia', 'albania', 'kazakhstan', 'qatar',
       'montenegro', 'cameroon', 'kenya', 'oman', 'mauritius', 'hungary',
       'chile', 'sri lanka', 'malta', 'cuba', 'indonesia', 'myanmar',
       '

In [277]:
# AR Index for Country-Author

nats_groups = sdg7_new[['PY', 'Unique Nationalities', 'citations sum']]
nats_groups = nats_groups.groupby('Unique Nationalities')

for nat_name, nat_group in nats_groups:
    py_groups = nat_group.groupby('PY')
    for py_name, py_group in py_groups:
        py_group = py_group.sort_values("citations sum", ascending=False)
        py_group = py_group.reset_index(drop=True)
#         print(py_group)
        
        ar_index = 0
        h = 1
        for index, row in py_group.iterrows():
            if (row['citations sum'] >= h) and (row['PY'] < 2021):
                ar_index += row['citations sum'] / (2021 - row['PY'])
            h += 1
        sdg7_new.loc[(sdg7_new['Unique Nationalities'] == py_group['Unique Nationalities'][0]) & (sdg7_new['PY']  == py_group['PY'][0]), 'ar index'] = np.sqrt(ar_index)
#         sdg7_new['rf index'] = np.where(sdg7_new['Unique Nationalities'] == py_group['Unique Nationalities'][0], sdg7_new['PY'] == py_group['PY'][0])
        print(py_group['PY'][0], py_group['Unique Nationalities'][0], np.sqrt(ar_index))

2019 albania 0.7071067811865476
2011 algeria 2.0248456731316584
2012 algeria 1.7320508075688772
2013 algeria 1.541103500742244
2014 algeria 4.358898943540674
2015 algeria 2.449489742783178
2016 algeria 3.0
2017 algeria 3.3541019662496847
2018 algeria 1.2909944487358056
2019 algeria 2.8284271247461903
2020 algeria 1.0
1998 arabia 2.440598643597598
1999 arabia 2.8444523358476395
2000 arabia 6.51737604225738
2001 arabia 1.0488088481701516
2002 arabia 4.285839597165482
2003 arabia 1.1055415967851334
2006 arabia 1.7320508075688772
2009 arabia 3.872983346207417
2011 arabia 2.3664319132398464
2013 arabia 1.9685019685029528
2015 arabia 2.345207879911715
2016 arabia 3.3166247903554
2017 arabia 1.8708286933869707
2019 arabia 5.244044240850758
2020 arabia 2.6457513110645907
2000 argentina 0.3779644730092272
2008 argentina 3.36269122990683
2012 argentina 2.3570226039551585
2015 argentina 1.4719601443879744
2016 argentina 1.6733200530681511
2018 argentina 4.041451884327381
2019 argentina 0.70710678

2010 netherlands 7.79277054244976
2011 netherlands 3.492849839314596
2012 netherlands 1.855921454276674
2013 netherlands 1.8371173070873836
2014 netherlands 7.755182598937123
2015 netherlands 5.916079783099616
2016 netherlands 2.932575659723036
2017 netherlands 2.8722813232690143
2018 netherlands 6.244997998398398
2019 netherlands 4.847679857416329
2020 netherlands 3.872983346207417
2003 nigeria 1.0274023338281628
2006 nigeria 2.503331114069145
2008 nigeria 0.9607689228305228
2011 nigeria 1.9235384061671346
2012 nigeria 2.943920288775949
2014 nigeria 3.0472470011002204
2015 nigeria 0.816496580927726
2016 nigeria 1.6733200530681511
2017 nigeria 3.5355339059327378
2018 nigeria 2.8284271247461903
2019 nigeria 5.385164807134504
2020 nigeria 3.7416573867739413
2003 norway 1.9436506316151
2007 norway 2.3754698783308412
2009 norway 1.2909944487358056
2011 norway 2.5298221281347035
2012 norway 1.1055415967851334
2014 norway 4.225771273642583
2017 norway 1.5811388300841898
2018 norway 7.7888809

2017 united states 10.35615758860399
2018 united states 12.675435561221029
2019 united states 8.154753215150045
2020 united states 5.830951894845301
2016 uruguay 2.4899799195977463
2017 vietnam 1.224744871391589
2019 vietnam 1.7320508075688772
2020 vietnam 1.0
2004 zealand 2.400980191995124
2006 zealand 1.0954451150103321
2008 zealand 1.1766968108291043
2009 zealand 1.9790570145063195
2011 zealand 2.4698178070456938
2012 zealand 0.8819171036881969
2015 zealand 2.972092416687835
2016 zealand 0.6324555320336759
2018 zealand 2.160246899469287
2019 zealand 2.5495097567963922
2020 zealand 1.4142135623730951


In [278]:
sdg7_new = sdg7_new.groupby(['PY', 'Unique Nationalities'], as_index=False).agg({'2 years window': 'sum', 'ar index': 'mean'})

In [279]:
sdg7_new

,PY,Unique Nationalities,2 years window,ar index
0,1991,india,1.0,0.707107
1,1991,united kingdom,5.0,0.516398
2,1992,brazil,2.0,0.454859
3,1994,china,1.0,0.192450
4,1994,france,1.0,0.471405
...,...,...,...,...
917,2020,ukraine,2.0,1.000000
918,2020,united kingdom,79.0,2.828427
919,2020,united states,248.0,5.830952
920,2020,vietnam,21.0,1.000000


In [280]:
# sdg7_new.head(5)
sdg7_new[sdg7_new.PY == 1996]

,PY,Unique Nationalities,2 years window,ar index
9,1996,belgium,1.0,0.282843
10,1996,china,3.0,1.200000
11,1996,germany,1.0,0.529150
12,1996,india,4.0,1.428286


##### 3.2 NULL CHECK

In [281]:
data = {'Publish Year': sdg7_new['PY'],
           'Unique Nationalities': sdg7_new['Unique Nationalities'],
           'AR Index': sdg7_new['ar index'],
           '2 Years Window': sdg7_new['2 years window']}
#            '5 Years Window': sdg7_new['5 year window']}
citations = pd.DataFrame(data)

In [282]:
citations #sdg7_new

,Publish Year,Unique Nationalities,AR Index,2 Years Window
0,1991,india,0.707107,1.0
1,1991,united kingdom,0.516398,5.0
2,1992,brazil,0.454859,2.0
3,1994,china,0.192450,1.0
4,1994,france,0.471405,1.0
...,...,...,...,...
917,2020,ukraine,1.000000,2.0
918,2020,united kingdom,2.828427,79.0
919,2020,united states,5.830952,248.0
920,2020,vietnam,1.000000,21.0


In [283]:
print("Null Check for Publish Year:", citations['Publish Year'].isnull().sum())
print("Null Check for Unique Nationalities:", citations['Unique Nationalities'].isnull().sum())
print("Null Check for AR Index:", citations['AR Index'].isnull().sum())
print("Null Check for 2 Years Window:", citations['2 Years Window'].isnull().sum())
# print("Null Check for 5 Years Window:", citations['5 Years Window'].isnull().sum())

Null Check for Publish Year: 0
Null Check for Unique Nationalities: 0
Null Check for AR Index: 0
Null Check for 2 Years Window: 0


# READ DATA & CLEANING - SDG_7
### 1. READ DATA
##### 1.1 UNITED THE COUNTRY NAME
##### 1.2 PUT ALL INDICAT DATA INTO ONE DATAFRAME
### 2. CLEANING
##### 2.1 COMPARE TWO DATA SET AND DROP THE 'CITATIONS['UNIQUE NATIONALITY']' THAT ARE NOR IN SDG DATA
##### 2.2 MERGE TWO DATASETS

In [284]:
sdg7_new = citations
sdg7_new['Publish Year'] = sdg7_new['Publish Year'].astype(str)
sdg7_new

,Publish Year,Unique Nationalities,AR Index,2 Years Window
0,1991,india,0.707107,1.0
1,1991,united kingdom,0.516398,5.0
2,1992,brazil,0.454859,2.0
3,1994,china,0.192450,1.0
4,1994,france,0.471405,1.0
...,...,...,...,...
917,2020,ukraine,1.000000,2.0
918,2020,united kingdom,2.828427,79.0
919,2020,united states,5.830952,248.0
920,2020,vietnam,1.000000,21.0


In [285]:
sdg7_new.columns

Index(['Publish Year', 'Unique Nationalities', 'AR Index', '2 Years Window'], dtype='object')

In [286]:
sdg7_new['Publish Year']

0      1991
1      1991
2      1992
3      1994
4      1994
       ... 
917    2020
918    2020
919    2020
920    2020
921    2020
Name: Publish Year, Length: 922, dtype: object

##### 1.1 UNITED THE COUNTRY NAME

In [287]:
#read in WDI data
only_wdi = pd.read_csv('data/WDI_onlydata.csv')
wdi_series = pd.read_csv('data/WDI_series.csv')

#get data for only energy indicators
eg_wdi = only_wdi[(only_wdi['Indicator Code'].str.startswith("EG."))]
eg_indicators = eg_wdi['Indicator Name'].unique().tolist() #all the energy indicators

# #abstarct data only has "energy" ABs from 1996, so we'll remove the data in wdi indicators before that (1960-1998, 1999 has NaNs)
eg_wdi = eg_wdi.drop(eg_wdi.iloc[:, 4:40],axis = 1)
eg_wdi = eg_wdi.drop('2021',axis = 1) #removing 2021, because very small num of citations

#convert country name to lower case to match ab data frame
eg_wdi['Country Name'] = eg_wdi['Country Name'].str.lower()

#get countries in list
ab = list(np.unique(sdg7_new['Unique Nationalities'].tolist())) #should be in lowercase
natwdi = list(np.unique(eg_wdi['Country Name'].tolist())) #should be in lowercase

#replace country name in AB data with country name in wdi data
in_ab = (list(set(ab) - set(natwdi))) #country names in ab but not in natwdi- we only want data for countries in ab data
for to_replace in in_ab:
    if eg_wdi['Country Name'].str.contains(to_replace).any():
        replacement = eg_wdi[eg_wdi['Country Name'].str.contains(to_replace)].iloc[0][0] 
        sdg7_new.loc[sdg7_new['Unique Nationalities'] == to_replace, 'Unique Nationalities'] = replacement
##################################################################

eg_wdi.head(4)
#get data from eg_wdi for countries in sdg7_new
#match countries in eg_wdi with countries in sdg3 AB
only_nat = list(np.unique(sdg7_new['Unique Nationalities'].tolist())) #after replacment nationalities
eg_wdi = eg_wdi[eg_wdi['Country Name'].str.lower().isin([x.lower() for x in only_nat])]
eg_wdi.head(4)

,Country Name,Country Code,Indicator Name,Indicator Code,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
72100,albania,ALB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,38.2,40.5,43.2,46.40,49.0,51.9,54.8,58.1,60.8,63.6,66.50,68.9,71.300000,73.15,74.600000,76.200000,77.800000,78.700000,79.8,80.70,81.3
72101,albania,ALB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,NaN,NaN,NaN,NaN,17.8,19.8,22.2,24.15,26.4,29.6,32.3,35.9,39.1,42.7,45.65,48.3,51.400000,53.80,55.700000,57.600000,59.050000,60.500000,62.5,63.25,64.6
72102,albania,ALB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,NaN,NaN,NaN,NaN,67.3,69.0,71.2,73.55,75.9,77.6,79.5,81.7,83.3,84.6,86.10,87.2,88.500000,89.30,89.900000,90.700000,91.100000,91.600000,92.2,92.30,92.9
72103,albania,ALB,Access to electricity (% of population),EG.ELC.ACCS.ZS,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.00,100.0,100.0,100.0,100.0,100.0,100.0,100.00,100.0,99.900002,100.00,99.949997,99.980003,99.889999,99.889999,100.0,100.00,100.0


In [288]:
eg_wdi.tail(4)

,Country Name,Country Code,Indicator Name,Indicator Code,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
375399,vietnam,VNM,GDP per unit of energy use (constant 2017 PPP ...,EG.GDP.PUSE.KO.PP.KD,9.858680,9.894637,9.819981,9.997748,10.147967,10.104953,9.845455,10.024299,9.712796,9.867481,10.301451,10.256161,10.134920,9.790574,9.385017,9.991590,10.357399,10.932122,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375400,vietnam,VNM,GDP per unit of energy use (PPP $ per kg of oi...,EG.GDP.PUSE.KO.PP,6.390947,6.524866,6.548520,6.761020,7.018082,7.145778,7.070781,7.341307,7.304125,7.653142,8.236255,8.421653,8.481710,8.246051,7.999468,8.693438,9.484979,10.129281,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376095,vietnam,VNM,Renewable electricity output (% of total elect...,EG.ELC.RNEW.ZS,70.868744,60.929333,51.157322,58.465979,54.783329,59.494250,50.838082,46.392181,38.559588,31.673997,33.843585,34.494389,35.481498,36.132251,29.140280,39.460181,45.019219,41.662002,42.981407,36.732058,NaN,NaN,NaN,NaN,NaN
376096,vietnam,VNM,Renewable energy consumption (% of total final...,EG.FEC.RNEW.ZS,62.866009,60.792374,59.499344,59.567003,57.730000,56.140000,52.200000,50.830000,45.770000,44.150000,44.250000,41.900000,39.260000,36.970000,34.580000,36.320000,37.900000,37.350000,36.640000,30.330000,22.48,25.02,21.81,18.65,NaN


In [289]:
eg_indicators

['Access to clean fuels and technologies for cooking (% of population)',
 'Access to clean fuels and technologies for cooking, rural (% of rural population)',
 'Access to clean fuels and technologies for cooking, urban (% of urban population)',
 'Access to electricity (% of population)',
 'Access to electricity, rural (% of rural population)',
 'Access to electricity, urban (% of urban population)',
 'Alternative and nuclear energy (% of total energy use)',
 'Combustible renewables and waste (% of total energy)',
 'Electric power consumption (kWh per capita)',
 'Electric power transmission and distribution losses (% of output)',
 'Electricity production from coal sources (% of total)',
 'Electricity production from hydroelectric sources (% of total)',
 'Electricity production from natural gas sources (% of total)',
 'Electricity production from nuclear sources (% of total)',
 'Electricity production from oil sources (% of total)',
 'Electricity production from oil, gas and coal sources

##### 1.2 PUT ALL INDICAT DATA INTO ONE DATAFRAME

In [290]:
Indicators = pd.DataFrame()

In [291]:
temp = eg_indicators[0]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
cooking_data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'cooking'])
cooking_data #this is tidy!

,Country Name,Year,cooking
0,albania,1996,NaN
1,albania,1997,NaN
2,albania,1998,NaN
3,albania,1999,NaN
4,albania,2000,38.2
...,...,...,...
2249,vietnam,2014,58.3
2250,vietnam,2015,59.6
2251,vietnam,2016,61.0
2252,vietnam,2017,62.3


In [292]:
temp = eg_indicators[1]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
rural_cooking_data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'rural_cooking'])
rural_cooking_data #this is tidy!

,Country Name,Year,rural_cooking
0,albania,1996,NaN
1,albania,1997,NaN
2,albania,1998,NaN
3,albania,1999,NaN
4,albania,2000,17.8
...,...,...,...
2249,vietnam,2014,45.8
2250,vietnam,2015,48.1
2251,vietnam,2016,49.9
2252,vietnam,2017,51.4


In [293]:
Indicators = pd.merge(cooking_data, rural_cooking_data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking
0,albania,1996,NaN,NaN
1,albania,1997,NaN,NaN
2,albania,1998,NaN,NaN
3,albania,1999,NaN,NaN
4,albania,2000,38.2,17.8
...,...,...,...,...
2249,vietnam,2014,58.3,45.8
2250,vietnam,2015,59.6,48.1
2251,vietnam,2016,61.0,49.9
2252,vietnam,2017,62.3,51.4


In [294]:
temp = eg_indicators[2]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'urban_cooking'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking
0,albania,1996,NaN,NaN,NaN
1,albania,1997,NaN,NaN,NaN
2,albania,1998,NaN,NaN,NaN
3,albania,1999,NaN,NaN,NaN
4,albania,2000,38.2,17.8,67.3
...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2
2250,vietnam,2015,59.6,48.1,82.3
2251,vietnam,2016,61.0,49.9,82.5
2252,vietnam,2017,62.3,51.4,82.6


In [295]:
temp = eg_indicators[3]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
electricity = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'electricy'])
Indicators = pd.merge(Indicators, electricity, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy
0,albania,1996,NaN,NaN,NaN,100.000000
1,albania,1997,NaN,NaN,NaN,100.000000
2,albania,1998,NaN,NaN,NaN,100.000000
3,albania,1999,NaN,NaN,NaN,100.000000
4,albania,2000,38.2,17.8,67.3,100.000000
...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997
2250,vietnam,2015,59.6,48.1,82.3,99.821304
2251,vietnam,2016,61.0,49.9,82.5,99.199997
2252,vietnam,2017,62.3,51.4,82.6,100.000000


In [296]:
temp = eg_indicators[4]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
rural_electricity = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'rural_electricy'])
Indicators = pd.merge(Indicators, rural_electricity, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000
...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000


In [297]:
temp = eg_indicators[5]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
urban_electricity = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'urban_electricy'])
Indicators = pd.merge(Indicators, urban_electricity, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000
...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000


In [298]:
temp = eg_indicators[6]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'nuclear_energy'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699
...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN


In [299]:
temp = eg_indicators[7]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'combustiable_renewable_waste'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960
...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN


In [300]:
temp = eg_indicators[8]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'electric_consumption'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413
...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN


In [301]:
temp = eg_indicators[9]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'eletric_loss'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895
...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN


In [302]:
temp = eg_indicators[10]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'coal'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN


In [303]:
temp = eg_indicators[11]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'hydroeletric'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000,95.720495
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000,96.174445
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000,96.320219
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000,97.329158
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000,96.149016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083,42.876796
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446,36.613975
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [304]:
temp = eg_indicators[12]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'natrual_gas'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000,95.720495,0.000000
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000,96.174445,0.000000
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000,96.320219,0.000000
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000,97.329158,0.000000
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000,96.149016,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083,42.876796,33.406656
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446,36.613975,33.210467
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [305]:
temp = eg_indicators[13]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'nuclear'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000,95.720495,0.000000,0.0
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000,96.174445,0.000000,0.0
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000,96.320219,0.000000,0.0
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000,97.329158,0.000000,0.0
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000,96.149016,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083,42.876796,33.406656,0.0
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446,36.613975,33.210467,NaN
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [306]:
temp = eg_indicators[14]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'oil'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000,95.720495,0.000000,0.0,4.279505
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000,96.174445,0.000000,0.0,3.825555
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000,96.320219,0.000000,0.0,3.679781
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000,97.329158,0.000000,0.0,2.670842
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000,96.149016,0.000000,0.0,3.850984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083,42.876796,33.406656,0.0,0.366854
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446,36.613975,33.210467,NaN,0.486029
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [307]:
temp = eg_indicators[15]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'oil_gas_coal'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000,95.720495,0.000000,0.0,4.279505,4.279505
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000,96.174445,0.000000,0.0,3.825555,3.825555
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000,96.320219,0.000000,0.0,3.679781,3.679781
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000,97.329158,0.000000,0.0,2.670842,2.670842
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000,96.149016,0.000000,0.0,3.850984,3.850984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083,42.876796,33.406656,0.0,0.366854,57.018593
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446,36.613975,33.210467,NaN,0.486029,63.267942
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [308]:
temp = eg_indicators[16]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'electric_from_renuable'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000,95.720495,0.000000,0.0,4.279505,4.279505,0.000000
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000,96.174445,0.000000,0.0,3.825555,3.825555,0.000000
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000,96.320219,0.000000,0.0,3.679781,3.679781,0.000000
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000,97.329158,0.000000,0.0,2.670842,2.670842,0.000000
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000,96.149016,0.000000,0.0,3.850984,3.850984,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083,42.876796,33.406656,0.0,0.366854,57.018593,0.104611
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446,36.613975,33.210467,NaN,0.486029,63.267942,0.118082
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [309]:
temp = eg_indicators[17]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'electric_from_renuable_kwh'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000,95.720495,0.000000,0.0,4.279505,4.279505,0.000000,0.0
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000,96.174445,0.000000,0.0,3.825555,3.825555,0.000000,0.0
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000,96.320219,0.000000,0.0,3.679781,3.679781,0.000000,0.0
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000,97.329158,0.000000,0.0,2.670842,2.670842,0.000000,0.0
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000,96.149016,0.000000,0.0,3.850984,3.850984,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083,42.876796,33.406656,0.0,0.366854,57.018593,0.104611,146000000.0
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446,36.613975,33.210467,NaN,0.486029,63.267942,0.118082,181000000.0
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [310]:
temp = eg_indicators[18]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'energy_import'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000,95.720495,0.000000,0.0,4.279505,4.279505,0.000000,0.0,6.291303
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000,96.174445,0.000000,0.0,3.825555,3.825555,0.000000,0.0,8.699857
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000,96.320219,0.000000,0.0,3.679781,3.679781,0.000000,0.0,18.412903
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000,97.329158,0.000000,0.0,2.670842,2.670842,0.000000,0.0,40.249672
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000,96.149016,0.000000,0.0,3.850984,3.850984,0.000000,0.0,45.002409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083,42.876796,33.406656,0.0,0.366854,57.018593,0.104611,146000000.0,NaN
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446,36.613975,33.210467,NaN,0.486029,63.267942,0.118082,181000000.0,NaN
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [311]:
temp = eg_indicators[19]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'energy_intensity'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000,95.720495,0.000000,0.0,4.279505,4.279505,0.000000,0.0,6.291303,NaN
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000,96.174445,0.000000,0.0,3.825555,3.825555,0.000000,0.0,8.699857,NaN
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000,96.320219,0.000000,0.0,3.679781,3.679781,0.000000,0.0,18.412903,NaN
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000,97.329158,0.000000,0.0,2.670842,2.670842,0.000000,0.0,40.249672,NaN
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000,96.149016,0.000000,0.0,3.850984,3.850984,0.000000,0.0,45.002409,4.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083,42.876796,33.406656,0.0,0.366854,57.018593,0.104611,146000000.0,NaN,5.12
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446,36.613975,33.210467,NaN,0.486029,63.267942,0.118082,181000000.0,NaN,5.13
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.53
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.38


In [312]:
temp = eg_indicators[20]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'energy_use(kg/capita)'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita)
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000,95.720495,0.000000,0.0,4.279505,4.279505,0.000000,0.0,6.291303,NaN,447.848870
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000,96.174445,0.000000,0.0,3.825555,3.825555,0.000000,0.0,8.699857,NaN,384.594958
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000,96.320219,0.000000,0.0,3.679781,3.679781,0.000000,0.0,18.412903,NaN,426.682819
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000,97.329158,0.000000,0.0,2.670842,2.670842,0.000000,0.0,40.249672,NaN,576.183632
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000,96.149016,0.000000,0.0,3.850984,3.850984,0.000000,0.0,45.002409,4.13,580.494764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083,42.876796,33.406656,0.0,0.366854,57.018593,0.104611,146000000.0,NaN,5.12,NaN
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446,36.613975,33.210467,NaN,0.486029,63.267942,0.118082,181000000.0,NaN,5.13,NaN
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.53,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.38,NaN


In [313]:
temp = eg_indicators[21]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'energy_use(kg/kGDP)'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP)
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000,95.720495,0.000000,0.0,4.279505,4.279505,0.000000,0.0,6.291303,NaN,447.848870,91.231290
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000,96.174445,0.000000,0.0,3.825555,3.825555,0.000000,0.0,8.699857,NaN,384.594958,87.401609
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000,96.320219,0.000000,0.0,3.679781,3.679781,0.000000,0.0,18.412903,NaN,426.682819,88.540404
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000,97.329158,0.000000,0.0,2.670842,2.670842,0.000000,0.0,40.249672,NaN,576.183632,105.241755
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000,96.149016,0.000000,0.0,3.850984,3.850984,0.000000,0.0,45.002409,4.13,580.494764,98.512659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083,42.876796,33.406656,0.0,0.366854,57.018593,0.104611,146000000.0,NaN,5.12,NaN,NaN
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446,36.613975,33.210467,NaN,0.486029,63.267942,0.118082,181000000.0,NaN,5.13,NaN,NaN
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.53,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.38,NaN,NaN


In [314]:
temp = eg_indicators[22]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'fossil_fual_comsumption'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000,95.720495,0.000000,0.0,4.279505,4.279505,0.000000,0.0,6.291303,NaN,447.848870,91.231290,45.453764
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000,96.174445,0.000000,0.0,3.825555,3.825555,0.000000,0.0,8.699857,NaN,384.594958,87.401609,41.449340
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000,96.320219,0.000000,0.0,3.679781,3.679781,0.000000,0.0,18.412903,NaN,426.682819,88.540404,45.294818
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000,97.329158,0.000000,0.0,2.670842,2.670842,0.000000,0.0,40.249672,NaN,576.183632,105.241755,58.238794
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000,96.149016,0.000000,0.0,3.850984,3.850984,0.000000,0.0,45.002409,4.13,580.494764,98.512659,58.715098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083,42.876796,33.406656,0.0,0.366854,57.018593,0.104611,146000000.0,NaN,5.12,NaN,NaN,NaN
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446,36.613975,33.210467,NaN,0.486029,63.267942,0.118082,181000000.0,NaN,5.13,NaN,NaN,NaN
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.53,NaN,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.38,NaN,NaN,NaN


In [315]:
temp = eg_indicators[23]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'GDP/Energy(constrant_2017)'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017)
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000,95.720495,0.000000,0.0,4.279505,4.279505,0.000000,0.0,6.291303,NaN,447.848870,91.231290,45.453764,10.961152
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000,96.174445,0.000000,0.0,3.825555,3.825555,0.000000,0.0,8.699857,NaN,384.594958,87.401609,41.449340,11.441437
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000,96.320219,0.000000,0.0,3.679781,3.679781,0.000000,0.0,18.412903,NaN,426.682819,88.540404,45.294818,11.294279
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000,97.329158,0.000000,0.0,2.670842,2.670842,0.000000,0.0,40.249672,NaN,576.183632,105.241755,58.238794,9.501932
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000,96.149016,0.000000,0.0,3.850984,3.850984,0.000000,0.0,45.002409,4.13,580.494764,98.512659,58.715098,10.150980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083,42.876796,33.406656,0.0,0.366854,57.018593,0.104611,146000000.0,NaN,5.12,NaN,NaN,NaN,NaN
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446,36.613975,33.210467,NaN,0.486029,63.267942,0.118082,181000000.0,NaN,5.13,NaN,NaN,NaN,NaN
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.53,NaN,NaN,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.38,NaN,NaN,NaN,NaN


In [316]:
temp = eg_indicators[24]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'GDP/Energy'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017),GDP/Energy
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000,95.720495,0.000000,0.0,4.279505,4.279505,0.000000,0.0,6.291303,NaN,447.848870,91.231290,45.453764,10.961152,6.654867
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000,96.174445,0.000000,0.0,3.825555,3.825555,0.000000,0.0,8.699857,NaN,384.594958,87.401609,41.449340,11.441437,7.066251
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000,96.320219,0.000000,0.0,3.679781,3.679781,0.000000,0.0,18.412903,NaN,426.682819,88.540404,45.294818,11.294279,7.080298
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000,97.329158,0.000000,0.0,2.670842,2.670842,0.000000,0.0,40.249672,NaN,576.183632,105.241755,58.238794,9.501932,6.025269
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000,96.149016,0.000000,0.0,3.850984,3.850984,0.000000,0.0,45.002409,4.13,580.494764,98.512659,58.715098,10.150980,6.650886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083,42.876796,33.406656,0.0,0.366854,57.018593,0.104611,146000000.0,NaN,5.12,NaN,NaN,NaN,NaN,NaN
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446,36.613975,33.210467,NaN,0.486029,63.267942,0.118082,181000000.0,NaN,5.13,NaN,NaN,NaN,NaN,NaN
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.53,NaN,NaN,NaN,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.38,NaN,NaN,NaN,NaN,NaN


In [317]:
temp = eg_indicators[25]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'renewable_output'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017),GDP/Energy,renewable_output
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000,95.720495,0.000000,0.0,4.279505,4.279505,0.000000,0.0,6.291303,NaN,447.848870,91.231290,45.453764,10.961152,6.654867,95.720495
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000,96.174445,0.000000,0.0,3.825555,3.825555,0.000000,0.0,8.699857,NaN,384.594958,87.401609,41.449340,11.441437,7.066251,96.174445
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000,96.320219,0.000000,0.0,3.679781,3.679781,0.000000,0.0,18.412903,NaN,426.682819,88.540404,45.294818,11.294279,7.080298,96.320219
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000,97.329158,0.000000,0.0,2.670842,2.670842,0.000000,0.0,40.249672,NaN,576.183632,105.241755,58.238794,9.501932,6.025269,97.329158
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000,96.149016,0.000000,0.0,3.850984,3.850984,0.000000,0.0,45.002409,4.13,580.494764,98.512659,58.715098,10.150980,6.650886,96.149016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083,42.876796,33.406656,0.0,0.366854,57.018593,0.104611,146000000.0,NaN,5.12,NaN,NaN,NaN,NaN,NaN,42.981407
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446,36.613975,33.210467,NaN,0.486029,63.267942,0.118082,181000000.0,NaN,5.13,NaN,NaN,NaN,NaN,NaN,36.732058
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.53,NaN,NaN,NaN,NaN,NaN,NaN
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.38,NaN,NaN,NaN,NaN,NaN,NaN


In [318]:
temp = eg_indicators[26]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'renewable_comsumption'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017),GDP/Energy,renewable_output,renewable_comsumption
0,albania,1996,NaN,NaN,NaN,100.000000,100.000000,100.000000,16.025289,21.232450,904.346640,48.261451,0.000000,95.720495,0.000000,0.0,4.279505,4.279505,0.000000,0.0,6.291303,NaN,447.848870,91.231290,45.453764,10.961152,6.654867,95.720495,51.636361
1,albania,1997,NaN,NaN,NaN,100.000000,100.000000,100.000000,14.372317,23.747598,694.664803,55.719204,0.000000,96.174445,0.000000,0.0,3.825555,3.825555,0.000000,0.0,8.699857,NaN,384.594958,87.401609,41.449340,11.441437,7.066251,96.174445,55.953349
2,albania,1998,NaN,NaN,NaN,100.000000,100.000000,100.000000,13.875295,20.559971,734.849913,62.419260,0.000000,96.320219,0.000000,0.0,3.679781,3.679781,0.000000,0.0,18.412903,NaN,426.682819,88.540404,45.294818,11.294279,7.080298,96.320219,49.983365
3,albania,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,20.236480,14.623496,1414.703784,25.584822,0.000000,97.329158,0.000000,0.0,2.670842,2.670842,0.000000,0.0,40.249672,NaN,576.183632,105.241755,58.238794,9.501932,6.025269,97.329158,42.718313
4,albania,2000,38.2,17.8,67.3,100.000000,100.000000,100.000000,19.677699,14.383960,1449.647413,27.249895,0.000000,96.149016,0.000000,0.0,3.850984,3.850984,0.000000,0.0,45.002409,4.13,580.494764,98.512659,58.715098,10.150980,6.650886,96.149016,41.360000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,58.3,45.8,82.2,99.199997,98.853424,99.900002,NaN,NaN,1423.699910,9.286712,23.245083,42.876796,33.406656,0.0,0.366854,57.018593,0.104611,146000000.0,NaN,5.12,NaN,NaN,NaN,NaN,NaN,42.981407,36.640000
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.000000,NaN,NaN,NaN,NaN,29.571446,36.613975,33.210467,NaN,0.486029,63.267942,0.118082,181000000.0,NaN,5.13,NaN,NaN,NaN,NaN,NaN,36.732058,30.330000
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.53,NaN,NaN,NaN,NaN,NaN,NaN,22.480000
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.38,NaN,NaN,NaN,NaN,NaN,NaN,25.020000


##### 2.1 COMPARE TWO DATA SET AND DROP THE 'CITATIONS['UNIQUE NATIONALITY']' THAT ARE NOR IN SDG DATA
##### 2.2 MERGE TWO DATASETS

In [319]:
Indicators.head(4)

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017),GDP/Energy,renewable_output,renewable_comsumption
0,albania,1996,NaN,NaN,NaN,100.0,100.0,100.0,16.025289,21.232450,904.346640,48.261451,0.0,95.720495,0.0,0.0,4.279505,4.279505,0.0,0.0,6.291303,NaN,447.848870,91.231290,45.453764,10.961152,6.654867,95.720495,51.636361
1,albania,1997,NaN,NaN,NaN,100.0,100.0,100.0,14.372317,23.747598,694.664803,55.719204,0.0,96.174445,0.0,0.0,3.825555,3.825555,0.0,0.0,8.699857,NaN,384.594958,87.401609,41.449340,11.441437,7.066251,96.174445,55.953349
2,albania,1998,NaN,NaN,NaN,100.0,100.0,100.0,13.875295,20.559971,734.849913,62.419260,0.0,96.320219,0.0,0.0,3.679781,3.679781,0.0,0.0,18.412903,NaN,426.682819,88.540404,45.294818,11.294279,7.080298,96.320219,49.983365
3,albania,1999,NaN,NaN,NaN,100.0,100.0,100.0,20.236480,14.623496,1414.703784,25.584822,0.0,97.329158,0.0,0.0,2.670842,2.670842,0.0,0.0,40.249672,NaN,576.183632,105.241755,58.238794,9.501932,6.025269,97.329158,42.718313


In [320]:
Indicators.tail(4)

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017),GDP/Energy,renewable_output,renewable_comsumption
2250,vietnam,2015,59.6,48.1,82.3,99.821304,99.730034,100.0,NaN,NaN,NaN,NaN,29.571446,36.613975,33.210467,NaN,0.486029,63.267942,0.118082,181000000.0,NaN,5.13,NaN,NaN,NaN,NaN,NaN,36.732058,30.33
2251,vietnam,2016,61.0,49.9,82.5,99.199997,98.778442,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.53,NaN,NaN,NaN,NaN,NaN,NaN,22.48
2252,vietnam,2017,62.3,51.4,82.6,100.000000,100.000000,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.38,NaN,NaN,NaN,NaN,NaN,NaN,25.02
2253,vietnam,2018,63.3,52.9,82.5,100.000000,100.000000,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.74,NaN,NaN,NaN,NaN,NaN,NaN,21.81


In [321]:
Indicators['Year'] = Indicators['Year'].astype(str)

## Indicators Interpolation

In [322]:
all_countries = list(np.unique(Indicators['Country Name'].tolist()))
all_years = list(np.unique(Indicators['Year'].tolist()))

In [323]:
cnt = 0
for i in all_countries:
    cnt += 1
    df = Indicators[Indicators['Country Name'] == i]
    country_yr = df[['Country Name','Year']]
    df.drop(['Country Name','Year'], axis=1, inplace=True)
    df.interpolate(method='linear', limit_direction='forward', limit_area='inside')
    
    df_new = pd.concat([country_yr, df], axis="columns")
    
    if cnt == 1:
        final_ind_df = df_new
    
    elif cnt >=2:
        final_ind_df = pd.concat([final_ind_df, df_new])

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [324]:
print('countries in final:',len(list(np.unique(final_ind_df['Country Name'].tolist()))))

print('countries in indicator:',len(all_countries))

countries in final: 98
countries in indicator: 98


In [325]:
final_ind_df.shape

(2254, 29)

In [326]:
Indicators.shape

(2254, 29)

In [327]:
#remove continents and regions from final_ind_df--only keep countries
remove = ['arab world', 'world','caribbean small states','central europe and the baltics','africa eastern and southern',
 'africa western and central','early-demographic dividend',
 'east asia & pacific',
 'east asia & pacific (excluding high income)',
 'east asia & pacific (ida & ibrd countries)','euro area',
 'europe & central asia',
 'europe & central asia (excluding high income)',
 'europe & central asia (ida & ibrd countries)',
 'european union','fragile and conflict affected situations','heavily indebted poor countries (hipc)','high income',
'ida & ibrd total',
 'ida blend',
 'ida only',
 'ida total', 'late-demographic dividend',
 'latin america & caribbean',
 'latin america & caribbean (excluding high income)',
 'latin america & the caribbean (ida & ibrd countries)','least developed countries: un classification','low & middle income',
 'low income',
 'lower middle income','middle east & north africa',
 'middle east & north africa (excluding high income)',
 'middle east & north africa (ida & ibrd countries)',
 'middle income','north america','post-demographic dividend',
 'pre-demographic dividend','pacific island small states','small states','south asia',
 'south asia (ida & ibrd)','sub-saharan africa',
 'sub-saharan africa (excluding high income)',
 'sub-saharan africa (ida & ibrd countries)',
         'not classified',
 'oecd members',
 'other small states','upper middle income']

final_ind_df = final_ind_df[~final_ind_df['Country Name'].isin(remove)]

In [328]:
all_countries = list(np.unique(final_ind_df['Country Name'].tolist()))
all_years = list(np.unique(final_ind_df['Year'].tolist()))

## % Change calculation on interpolated data

In [329]:
cnt = 0
for i in all_countries:
    cnt += 1
    df = final_ind_df[final_ind_df['Country Name'] == i]
    country_yr = df[['Country Name','Year']]
    df.drop(['Country Name','Year'], axis=1, inplace=True)
    
    df = df.pct_change() #gives (new value-old value)/old value
    df = df.multiply(100, axis="index") #multiplies change ratio by 100 to get percent
    
    df_new = pd.concat([country_yr, df], axis="columns")
    
    if cnt == 1:
        pc_change_df = df_new
    
    elif cnt >=2:
        pc_change_df = pd.concat([pc_change_df, df_new])

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/2897285658.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/2897285658.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/2897285658.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

## log Change

In [330]:
cnt = 0
for i in all_countries:
    cnt += 1
    df = final_ind_df[final_ind_df['Country Name'] == i]
    country_yr = df[['Country Name','Year']]
    df.drop(['Country Name','Year'], axis=1, inplace=True)
    
    df = np.log(df)
    df = df.diff()
#     df = df.pct_change() #gives (new value-old value)/old value
#     df = df.multiply(100, axis="index") #multiplies change ratio by 100 to get percent
    
    df_new = pd.concat([country_yr, df], axis="columns")
    
    if cnt == 1:
        log_diff_df = df_new
    
    elif cnt >=2:
        log_diff_df = pd.concat([log_diff_df, df_new])

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/2493170385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/2493170385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarnin

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/2493170385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/2493170385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/2493170385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/2493170385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9306/2493170385.py:6: SettingWithC

In [331]:
log_diff_df

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017),GDP/Energy,renewable_output,renewable_comsumption
0,albania,1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,albania,1997,NaN,NaN,NaN,0.000000,0.000000,0.00000,-0.108864,0.111951,-0.263783,0.143692,NaN,0.004731,NaN,NaN,-0.112134,-0.112134,NaN,NaN,0.324138,NaN,-0.152265,-0.042884,-0.092224,0.042884,0.059982,0.004731,0.080292
2,albania,1998,NaN,NaN,NaN,0.000000,0.000000,0.00000,-0.035194,-0.144135,0.056237,0.113549,NaN,0.001515,NaN,NaN,-0.038850,-0.038850,NaN,NaN,0.749745,NaN,0.103850,0.012945,0.088721,-0.012945,0.001986,0.001515,-0.112828
3,albania,1999,NaN,NaN,NaN,0.000000,0.000000,0.00000,0.377377,-0.340717,0.655009,-0.891875,NaN,0.010420,NaN,NaN,-0.320459,-0.320459,NaN,NaN,0.782050,NaN,0.300385,0.172801,0.251359,-0.172801,-0.161354,0.010420,-0.157063
4,albania,2000,NaN,NaN,NaN,0.000000,0.000000,0.00000,-0.028001,-0.016516,0.024400,0.063050,NaN,-0.012199,NaN,NaN,0.365935,0.365935,NaN,NaN,0.111614,NaN,0.007454,-0.066075,0.008145,0.066075,0.098788,-0.012199,-0.032313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,vietnam,2014,0.036685,0.042370,0.007326,0.006524,0.009909,-0.00064,NaN,NaN,0.108858,0.009639,-0.053094,0.031528,-0.002382,NaN,0.078806,-0.022876,-0.102912,0.006873,NaN,0.054176,NaN,NaN,NaN,NaN,NaN,0.031178,-0.019192
2250,vietnam,2015,0.022053,0.048998,0.001216,0.006244,0.008829,0.00100,NaN,NaN,NaN,NaN,0.240716,-0.157901,-0.005890,NaN,0.281304,0.104001,0.121135,0.214890,NaN,0.001951,NaN,NaN,NaN,NaN,NaN,-0.157118,-0.189003
2251,vietnam,2016,0.023218,0.036739,0.002427,-0.006244,-0.009587,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.124384,NaN,NaN,NaN,NaN,NaN,NaN,-0.299511
2252,vietnam,2017,0.021088,0.029617,0.001211,0.008032,0.012291,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.033673,NaN,NaN,NaN,NaN,NaN,NaN,0.107049


In [332]:
pc_change_df.shape

(2254, 29)

In [333]:
sdg7_new

,Publish Year,Unique Nationalities,AR Index,2 Years Window
0,1991,india,0.707107,1.0
1,1991,united kingdom,0.516398,5.0
2,1992,brazil,0.454859,2.0
3,1994,china,0.192450,1.0
4,1994,france,0.471405,1.0
...,...,...,...,...
917,2020,ukraine,1.000000,2.0
918,2020,united kingdom,2.828427,79.0
919,2020,united states,5.830952,248.0
920,2020,vietnam,1.000000,21.0


In [334]:
sdg7_new = pd.DataFrame(data=sdg7_new)
sdg7_new = sdg7_new.groupby(['Publish Year', 'Unique Nationalities'], as_index=False).agg({'2 Years Window': 'sum', 'AR Index': 'mean'})
sdg7_new.rename(columns={'Publish Year': 'Year', 'Unique Nationalities': 'Country Name','2 Years Window': '2 years window', 'AR Index': 'ar index ca'}, inplace=True)
sdg7_new['Year'] = sdg7_new['Year'].astype(str)
sdg7_new

,Year,Country Name,2 years window,ar index ca
0,1991,india,1.0,0.707107
1,1991,united kingdom,5.0,0.516398
2,1992,brazil,2.0,0.454859
3,1994,china,1.0,0.192450
4,1994,france,1.0,0.471405
...,...,...,...,...
917,2020,ukraine,2.0,1.000000
918,2020,united arab emirates,20.0,2.000000
919,2020,united kingdom,79.0,2.828427
920,2020,united states,248.0,5.830952


In [335]:
final = pd.merge(sdg7_new, pc_change_df,  how='left', on=['Country Name', 'Year'])
final.to_csv('data/pc_change_citations_ca_df.csv')

In [336]:
final = pd.merge(sdg7_new, log_diff_df,  how='left', on=['Country Name', 'Year'])
final.to_csv('data/log_diff_citations_ca_df.csv')

## Calculate Distance Measure on all countries and only OECD countries interpolated data

In [337]:
inds = final_ind_df.columns.tolist()[2:]

In [338]:
inds

['cooking',
 'rural_cooking',
 'urban_cooking',
 'electricy',
 'rural_electricy',
 'urban_electricy',
 'nuclear_energy',
 'combustiable_renewable_waste',
 'electric_consumption',
 'eletric_loss',
 'coal',
 'hydroeletric',
 'natrual_gas',
 'nuclear',
 'oil',
 'oil_gas_coal',
 'electric_from_renuable',
 'electric_from_renuable_kwh',
 'energy_import',
 'energy_intensity',
 'energy_use(kg/capita)',
 'energy_use(kg/kGDP)',
 'fossil_fual_comsumption',
 'GDP/Energy(constrant_2017)',
 'GDP/Energy',
 'renewable_output',
 'renewable_comsumption']

In [339]:
list(np.unique(eg_wdi['Indicator Name'].tolist()))

['Access to clean fuels and technologies for cooking (% of population)',
 'Access to clean fuels and technologies for cooking, rural (% of rural population)',
 'Access to clean fuels and technologies for cooking, urban (% of urban population)',
 'Access to electricity (% of population)',
 'Access to electricity, rural (% of rural population)',
 'Access to electricity, urban (% of urban population)',
 'Alternative and nuclear energy (% of total energy use)',
 'Combustible renewables and waste (% of total energy)',
 'Electric power consumption (kWh per capita)',
 'Electric power transmission and distribution losses (% of output)',
 'Electricity production from coal sources (% of total)',
 'Electricity production from hydroelectric sources (% of total)',
 'Electricity production from natural gas sources (% of total)',
 'Electricity production from nuclear sources (% of total)',
 'Electricity production from oil sources (% of total)',
 'Electricity production from oil, gas and coal sources

In [340]:
final_ind_df.shape

(2254, 29)

In [341]:
final_ind_df.head(5)

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017),GDP/Energy,renewable_output,renewable_comsumption
0,albania,1996,NaN,NaN,NaN,100.0,100.0,100.0,16.025289,21.232450,904.346640,48.261451,0.0,95.720495,0.0,0.0,4.279505,4.279505,0.0,0.0,6.291303,NaN,447.848870,91.231290,45.453764,10.961152,6.654867,95.720495,51.636361
1,albania,1997,NaN,NaN,NaN,100.0,100.0,100.0,14.372317,23.747598,694.664803,55.719204,0.0,96.174445,0.0,0.0,3.825555,3.825555,0.0,0.0,8.699857,NaN,384.594958,87.401609,41.449340,11.441437,7.066251,96.174445,55.953349
2,albania,1998,NaN,NaN,NaN,100.0,100.0,100.0,13.875295,20.559971,734.849913,62.419260,0.0,96.320219,0.0,0.0,3.679781,3.679781,0.0,0.0,18.412903,NaN,426.682819,88.540404,45.294818,11.294279,7.080298,96.320219,49.983365
3,albania,1999,NaN,NaN,NaN,100.0,100.0,100.0,20.236480,14.623496,1414.703784,25.584822,0.0,97.329158,0.0,0.0,2.670842,2.670842,0.0,0.0,40.249672,NaN,576.183632,105.241755,58.238794,9.501932,6.025269,97.329158,42.718313
4,albania,2000,38.2,17.8,67.3,100.0,100.0,100.0,19.677699,14.383960,1449.647413,27.249895,0.0,96.149016,0.0,0.0,3.850984,3.850984,0.0,0.0,45.002409,4.13,580.494764,98.512659,58.715098,10.150980,6.650886,96.149016,41.360000


In [342]:
target_dict = {'electricy':100, 'cooking':97.50,'renewable_comsumption':25.18, 'renewable_output':38.65,
               'energy_use(kg/capita)':18949.33, 'rural_cooking':100,'urban_cooking':100,
               'rural_electricy':100,'urban_electricy':100, 'fossil_fual_comsumption':0, 'coal':0, 
               'oil_gas_coal':0}

dist_calc_ls = []
yr_ls = []
country_ls = []
# ind_ls = []

count = 0
for ind in inds:
    if ind in target_dict:
        count += 1
        ind_df = final_ind_df[['Country Name', 'Year', ind]] #df by indicator

        for yr in all_years:
            yr_df = ind_df[ind_df.Year == yr] #df by year for that indicator
            
            SD = np.std(yr_df[ind]) #standard dev for all countries in that year for this indicator
            
#             N = len(list(np.unique((yr_df['Country Name']).tolist()))) #total countries in that year for thos indicator
            
            for country in all_countries:
                country_val = list(yr_df[yr_df['Country Name'] == country][ind])[0] #gets the value for that ind, in that yr, for that country
                
                if SD == 0: #to avoid divsion by zero error
                    dist_calc = float("nan") #assign dist_calc as nan

                else:
                    calc = (target_dict[ind]-country_val)/SD #distance of country from OECD target divided by standard dev
                    dist_calc = (max(calc,0)) #max between calc and 0, multiplied by 1/N
                
                #get all values for this indicator
                dist_calc_ls.append(dist_calc)
                yr_ls.append(yr)
                country_ls.append(country)
        
        #put list of values in dataframe
        add_df = pd.DataFrame(list(zip(country_ls, yr_ls,dist_calc_ls)),
                   columns =['Country Name', 'Year', ind])

        if count == 1: #if it's the first dataframe, final df is add_df
            dist_df = add_df

        elif count >= 2: #when a dataframe exists, merge new indicator calcs to that df
            dist_df = (pd.merge(dist_df, add_df, how='left', on=['Country Name','Year']))

        #assign lists to empty list to get new values for next ind
        dist_calc_ls = []
        country_ls = []
        yr_ls = []

In [343]:
dist_df.sort_values(by=['Country Name', 'Year'])

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,coal,oil_gas_coal,energy_use(kg/capita),fossil_fual_comsumption,renewable_output,renewable_comsumption
0,albania,1996,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,6.806574,0.0,0.000000,0.000000
98,albania,1997,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,6.538402,0.0,0.000000,0.000000
196,albania,1998,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,6.448713,0.0,0.000000,0.000000
294,albania,1999,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,6.243335,0.0,0.000000,0.000000
392,albania,2000,1.653236,2.050303,1.037903,0.000000,0.000000,0.000000,0.0,0.0,6.394209,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,vietnam,2014,1.246644,1.489186,0.695455,0.039553,0.044250,0.010697,0.0,0.0,NaN,NaN,0.000000,0.000000
1959,vietnam,2015,1.220096,1.438721,0.705421,0.009066,0.011304,0.000000,0.0,0.0,NaN,NaN,0.065768,0.000000
2057,vietnam,2016,1.190571,1.402342,0.711139,0.044306,0.055045,0.000000,NaN,NaN,NaN,NaN,NaN,0.110586
2155,vietnam,2017,1.157238,1.372815,0.721349,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0.006649


In [344]:
#for oecd only
oecd = ['australia','austria','belgium','canada','czechia','chile','colombia','denmark','estonia','finland','france', 'germany','greece','hungary','iceland','ireland','israel','italy','japan','korea, rep.','latvia','lithuania','luxembourg','mexico','netherlands','new zealand','norway','poland','portugal','slovenia', 'spain','sweden','switzerland','turkiye','united kingdom','united states']


#with oecd data
oecd_data = final_ind_df[final_ind_df['Country Name'].isin(oecd)].reset_index(drop=True)

target_dict = {'electricy':100, 'cooking':97.50,'renewable_comsumption':25.18, 'renewable_output':38.65,
               'energy_use(kg/capita)':18949.33, 'rural_cooking':100,'urban_cooking':100,
               'rural_electricy':100,'urban_electricy':100, 'fossil_fual_comsumption':0, 'coal':0, 
               'oil_gas_coal':0}

dist_calc_ls = []
yr_ls = []
country_ls = []
# ind_ls = []

count = 0
for ind in inds:
    if ind in target_dict:
        count += 1
        ind_df = oecd_data[['Country Name', 'Year', ind]] #df by indicator

        for yr in all_years:
            yr_df = ind_df[ind_df.Year == yr] #df by year for that indicator
            
            SD = np.std(yr_df[ind]) #standard dev for all countries in that year for this indicator
            
#             N = len(list(np.unique((yr_df['Country Name']).tolist()))) #total countries in that year for thos indicator
            
            for country in oecd:
                
#                 print(ind, yr, country)
                country_val = list(yr_df[yr_df['Country Name'] == country][ind])[0] #gets the value for that ind, in that yr, for that country
                
                if SD == 0: #to avoid divsion by zero error
                    dist_calc = float("nan") #assign dist_calc as nan

                else:
                    calc = (target_dict[ind]-country_val)/SD #distance of country from OECD target divided by standard dev
                    dist_calc = (max(calc,0)) #max between calc and 0, multiplied by 1/N
                
                #get all values for this indicator
                dist_calc_ls.append(dist_calc)
                yr_ls.append(yr)
                country_ls.append(country)
        
        #put list of values in dataframe
        add_df = pd.DataFrame(list(zip(country_ls, yr_ls,dist_calc_ls)),
                   columns =['Country Name', 'Year', ind])

        if count == 1: #if it's the first dataframe, final df is add_df
            dist_oecd_df = add_df

        elif count >= 2: #when a dataframe exists, merge new indicator calcs to that df
            dist_oecd_df = (pd.merge(dist_oecd_df, add_df, how='left', on=['Country Name','Year']))

        #assign lists to empty list to get new values for next ind
        dist_calc_ls = []
        country_ls = []
        yr_ls = []

In [345]:
dist_oecd_df.shape

(828, 14)

In [346]:
dist_oecd_df

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,coal,oil_gas_coal,energy_use(kg/capita),fossil_fual_comsumption,renewable_output,renewable_comsumption
0,australia,1996,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,6.873934,0.0,0.979812,1.149315
1,austria,1996,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,7.793363,0.0,0.000000,0.080532
2,belgium,1996,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,6.801476,0.0,1.262333,1.690477
3,canada,1996,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,5.525149,0.0,0.000000,0.253672
4,czechia,1996,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,7.503047,0.0,1.169216,1.404652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,sweden,2018,0.0000,0.000000,0.000000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.000000
824,switzerland,2018,0.0000,0.000000,0.000000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.059451
825,turkiye,2018,0.8017,1.621636,0.382973,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.809866
826,united kingdom,2018,0.0000,0.000000,0.000000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.860824


#### Merge

In [347]:
sdg7_new

,Year,Country Name,2 years window,ar index ca
0,1991,india,1.0,0.707107
1,1991,united kingdom,5.0,0.516398
2,1992,brazil,2.0,0.454859
3,1994,china,1.0,0.192450
4,1994,france,1.0,0.471405
...,...,...,...,...
917,2020,ukraine,2.0,1.000000
918,2020,united arab emirates,20.0,2.000000
919,2020,united kingdom,79.0,2.828427
920,2020,united states,248.0,5.830952


In [348]:
final = pd.merge(sdg7_new, dist_df,  how='left', on=['Country Name','Year'])
final.to_csv('data/dist_calc_citations_ca_df.csv')

In [349]:
final = pd.merge(sdg7_new, dist_oecd_df,  how='left', on=['Country Name','Year'])
final.to_csv('data/dist_calc_oecd_citations_ca_df.csv')

# Extra Cells

In [224]:
# all_countries = list(np.unique(Indicators['Country Name'].tolist()))
# all_years = list(np.unique(Indicators['Year'].tolist()))

In [125]:
# uniq_nats_groups = Indicators
# uniq_nats_groups = uniq_nats_groups.set_index(['Country Name', 'Year'])
# uniq_nats_groups = (pd.concat([uniq_nats_groups, pd.DataFrame(index=['Country Name', 'Year'])]).interpolate(method='linear', limit_direction='forward', limit_area='inside'))
# uniq_nats_groups = uniq_nats_groups.reset_index()
# uniq_nats_groups.head(25)

In [225]:
# uniq_nats_groups = Indicators.groupby('Country Name')
# for nat_name, nat_group in uniq_nats_groups:
#     print(nat_name)
#     nat_group = nat_group.set_index(['Country Name', 'Year'])
#     nat_group = nat_group.interpolate(method='linear', limit_direction='forward', limit_area='inside')
#     for column_index, column in nat_group.iteritems():
# #         print(prev_cell)
#         count = 0
#         for cell in range(len(column)):
#             if count != 0:
#                 if (not np.isnan(column[count])) and (not np.isnan(column[cell])):
# #                     print(cell, prev_cell)
#                     cell = np.log(cell) - np.log(prev_cell)
# #                     print(cell)
#                 else:
#                     prev_cell = cell
#             count += 1
#         print(column)
#     nat_group = nat_group.reset_index()
#     nat_group = nat_group.reindex(nat_group.index.union('Country Name', 'Year')).sort_index(ascending=False).interpolate(method='linear', limit_direction='forward', limit_area='inside')
#     for (columnName, columnData) in nat_group.iteritems():
#         if (columnName != 'Country Name') and (columnName != 'Year'):
#             columnData = columnData.interpolate()
#         for index, row in nat_group.iterrows():
#             Indicators.loc[(Indicators['Country Name'] == row['Country Name']) & (Indicators['Year']  == py_group['PY'][0]), 'ar index'] = np.sqrt(ar_index)
#     Indicators.set_index(['Country Name, Year'], inplace=True)
#     Indicators.update(.set_index(['Country Name', 'Year']))
#     df1.reset_index()
#     print(nat_group)

In [226]:
# for i in range(len(all_countries)):

# #23x29
#     df_country = Indicators[Indicators['Country Name'] == all_countries[i]] #this would go in a loop that iterates over every country
#     for i in df_country.columns:
#         if len(df_country) == df_country[i].isna().sum():
#             df_country.drop(i, axis=1, inplace=True)

#         else:
#             first_ind_df = df_country.index[0] #first index of the data frame
#             first_not_nan_ind = df_country[i].first_valid_index() #first non nan value's index in df--column specific
#             list_top_nans = list(np.arange(first_ind_df,first_not_nan_ind,1))

#             last_ind_df = df_country.index[-1] #last index of the data frame
#             last_not_nan_ind = df_country[i].last_valid_index() #last non nan value's index in df--column specific
#             list_bottom_nans = list(np.arange(last_not_nan_ind+1,last_ind_df+1,1))

#             if len(list_top_nans) != 0 and len(list_bottom_nans) != 0: #when there are nans at start and end
#                 df_country.drop(index=list(np.arange(first_ind_df,first_not_nan_ind,1)),inplace=True) #drop first few nans using nan indices
#                 df_country.drop(index=list(np.arange(last_not_nan_ind+1,last_ind_df+1,1)),inplace=True) #drop last few nans using nan indices

#             elif len(list_top_nans) == 0 and len(list_bottom_nans) != 0: #when only nans at end
#                 df_country.drop(index=list(np.arange(last_not_nan_ind+1,last_ind_df+1,1)),inplace=True) #drop last few nans using nan indices

#             #not working
#             elif len(list_top_nans) != 0 and len(list_bottom_nans) == 0: #when only nans at start
#                 df_country.drop(index=list(np.arange(first_ind_df,first_not_nan_ind,1)),inplace=True) #drop first few nans using nan indices
#     print(df_country)